**Tabla de contenido**

- [Technical requirements](#Technical-requirements)
- [Comprendiendo el problema de la previsión](#Comprendiendo-el-problema-de-la-prevision)
- [Diseñando nuestro servicio de pronósticos](#Disenando-nuestro-servicio-de-pronosticos)
- [Seleccionando las herramientas](#Seleccionando-las-herramientas)
- [Training at scale](#Training-at-scale)
- [Sirviendo los modelos con FastAPI](#Sirviendo-los-modelos-con-FastAPI)
- [Esquemas de respuesta y solicitud](#Esquemas-de-respuesta-y-solicitud)
- [Gestión de modelos en su microservicio](#Gestion-de-modelos-en-su-microservicio)
- [Uniéndolo todo](#Uniendolo-todo)
- [Contenerizar y desplegar en Kubernetes](#Contenerizar-y-desplegar-en-Kubernetes)
    - [Contenerizar la aplicación](#Contenerizar-la-aplicacion)
    - [Scaling up with Kubernetes](#Scaling-up-with-Kubernetes)
    - [Deployment strategies](#Deployment-strategies)
- [Resumen](#Resumen)

Este capítulo tratará de reunir algunos de los conocimientos que hemos adquirido en el libro hasta ahora con un ejemplo realista. Esto se basará en uno de los escenarios presentados en el Capítulo 1, Introducción a la Ingeniería de ML, donde se nos pedía construir un servicio de pronóstico para las ventas de artículos en la tienda. Discutiremos el escenario con cierto detalle y esbozaremos las decisiones clave que deben tomarse para que una solución sea una realidad, antes de mostrar cómo podemos emplear los procesos, herramientas y técnicas que hemos aprendido a lo largo de este libro para resolver partes clave del problema desde la perspectiva de la ingeniería de ML. Al final de este capítulo, deberías tener una visión clara de cómo construir tus propios microservicios de ML para resolver una variedad de problemas empresariales.

En este capítulo, cubriremos los siguientes temas:
- Comprender el problema de pronóstico
- Diseñar nuestro servicio de pronóstico
- Seleccionar las herramientas
- Capacitación a gran escala
- Servir los modelos con FastAPI
- Contenerizar y desplegar en Kubernetes

Cada tema nos brindará la oportunidad de recorrer las diferentes decisiones que debemos tomar como ingenieros trabajando en una entrega compleja de ML. ¡Esto nos proporcionará una referencia útil cuando salgamos y hagamos esto en el mundo real!

Con eso, ¡comencemos y construyamos un microservicio de previsión!

# Technical requirements

Los ejemplos de código en este capítulo serán más fáciles de seguir si tienes lo siguiente instalado y funcionando en tu máquina:

- Postman u otra herramienta de desarrollo de API
- Un gestor de clúster local de Kubernetes como minikube o kind
- La herramienta de línea de comandos de Kubernetes, kubectl

Hay varios archivos .yml de diferentes entornos conda dentro de la carpeta Chapter08 en el repositorio de GitHub del libro para los ejemplos técnicos, ya que hay algunos subcomponentes diferentes. Estos son:

- mlewp-capítulo08-entrenar: Esto especifica el entorno para ejecutar los scripts de entrenamiento.
- mlewp-capítulo08-servir: Esto especifica el entorno para la construcción del servicio web local FastAPI.
- mlewp-capítulo08-registrar: Esto proporciona la especificación del entorno para ejecutar el servidor de seguimiento MLflow.

En cada caso, crea el entorno Conda, como de costumbre, con:

- conda env create –f <ENVIRONMENT_NAME>.yml

Los ejemplos de Kubernetes en este capítulo también requieren cierta configuración del clúster y de los servicios que vamos a desplegar; estos se encuentran en la carpeta Chapter08/forecast bajo diferentes archivos .yml. Si estás usando kind, puedes crear un clúster con una configuración sencilla ejecutando:

- kind create cluster

O puedes usar uno de los archivos de configuración .yaml proporcionados en el repositorio:

- kind create cluster --config cluster-config-ch08.yaml

Minikube no ofrece una opción para leer una configuración de clúster .yaml como kind, por lo que, en su lugar, simplemente debes ejecutar:

- minikube start

para desplegar su clúster local.

# Comprendiendo el problema de la prevision

En el Capítulo 1, Introducción a la Ingeniería de ML, consideramos el ejemplo de un equipo de ML que ha sido encargado de proporcionar pronósticos de artículos a nivel de tiendas individuales en un negocio minorista. Los usuarios comerciales ficticios tenían los siguientes requisitos:

- Los pronósticos deberían mostrarse y ser accesibles a través de un panel basado en la web.
- El usuario debería poder solicitar pronósticos actualizados si es necesario.
- Los pronósticos deberían realizarse a nivel de tiendas individuales.
- Los usuarios estarán interesados en sus propias regiones/tiendas en cualquier sesión y no se preocuparán por las tendencias globales.
- El número de solicitudes de pronósticos actualizados en cualquier sesión será pequeño.

Dadas estas necesidades, podemos trabajar con el negocio para crear las siguientes historias de usuario, las cuales podemos ingresar en una herramienta como Jira, tal como se explica en el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático. Algunos ejemplos de historias de usuario que cubren estas necesidades serían los siguientes:

- `Historia de Usuario 1`: Como planificador logístico local, quiero iniciar sesión en un panel por la mañana a las 09:00 y poder ver las previsiones de demanda de artículos a nivel de tienda para los próximos días, para poder entender la demanda de transporte con antelación.

- `Historia de usuario 2`: Como planificador logístico local, quiero poder solicitar una actualización de mi pronóstico si veo que está desactualizado. Quiero que el nuevo pronóstico se devuelva en menos de 5 minutos para poder tomar decisiones sobre la demanda de transporte de manera efectiva.

- `Historia de Usuario 3:` Como planificador logístico local, quiero poder filtrar las previsiones para tiendas específicas para poder entender qué tiendas están generando demanda y utilizar esto en la toma de decisiones.

Estas historias de usuario son muy importantes para el desarrollo de la solución en su conjunto. Como estamos enfocados en los aspectos de ingeniería de aprendizaje automático del problema, ahora podemos profundizar en lo que esto significa para la construcción de la solución.

Por ejemplo, el deseo de poder ver pronósticos de la demanda de artículos a nivel de tienda puede traducirse bastante bien en algunos requisitos técnicos para la parte de ML de la solución. Esto nos indica que la variable objetivo será el número de artículos requeridos en un día particular. Nos dice que nuestro modelo o modelos de ML deben ser capaces de trabajar a nivel de tienda, por lo que o tenemos un modelo por tienda o el concepto de la tienda puede incorporarse como algún tipo de característica.

De manera similar, el requisito de que el usuario quiera poder solicitar una actualización de mi pronóstico si veo que está desactualizado ... Quiero que el nuevo pronóstico se obtenga en menos de cinco minutos, lo que impone un claro requisito de latencia en el entrenamiento. No podemos construir algo que tarde días en reentrenarse, por lo que esto puede sugerir que un modelo único construido a partir de todos los datos puede no ser la mejor solución.

Finalmente, la solicitud de poder filtrar pronósticos para tiendas específicas nuevamente respalda la idea de que lo que construyamos debe utilizar algún tipo de identificador de tienda en los datos, pero no necesariamente como una característica para el algoritmo. Por lo tanto, podríamos querer empezar a pensar en una lógica de aplicación que tome una solicitud del pronóstico para una tienda específica, identificada por este ID de tienda, y luego el modelo de ML y el pronóstico se recuperen solo para esa tienda mediante algún tipo de búsqueda o recuperación que use este ID como filtro.

Al recorrer este proceso, podemos ver cómo solo unas pocas líneas de requisitos nos han permitido comenzar a desarrollar cómo abordaremos el problema en la práctica. Algunas de estas ideas y otras podrían consolidarse tras un poco de lluvia de ideas entre nuestro equipo para el proyecto en una tabla como la de la Tabla 8.1:

| Historia de Usuario | Detalles | Requisitos Técnicos |
|----------------------|-----------|---------------------|
| **1** | Como planificador logístico local, quiero iniciar sesión en un panel a las 09:00 de la mañana y poder ver los pronósticos de demanda de artículos a nivel de tienda para los próximos días, de modo que pueda comprender la demanda de transporte con anticipación. | • Variable objetivo = demanda de artículos. <br> • Horizonte de pronóstico = 1–7 días. <br> • Acceso por API para un panel o solución de visualización. |
| **2** | Como planificador logístico local, quiero poder solicitar una actualización de mi pronóstico si veo que está desactualizado. Quiero que el nuevo pronóstico se genere en menos de 5 minutos para poder tomar decisiones sobre la demanda de transporte de manera efectiva. | • Reentrenamiento ligero. <br> • Modelo por tienda. |
| **3** | Como planificador logístico local, quiero poder filtrar los pronósticos para tiendas específicas para entender qué tiendas están impulsando la demanda y usar esta información en la toma de decisiones. | • Modelo por tienda. |


# Disenando nuestro servicio de pronosticos

Los requisitos en la sección Comprender el problema de pronóstico son las definiciones de los objetivos que necesitamos alcanzar, pero no son el método para lograrlos. Aprovechando nuestra comprensión del diseño y la arquitectura del Capítulo 5, Patrones y Herramientas de Despliegue, podemos comenzar a desarrollar nuestro diseño.

Primero, debemos confirmar qué tipo de diseño debemos desarrollar. Dado que necesitamos solicitudes dinámicas, tiene sentido seguir la arquitectura de microservicios discutida en el Capítulo 5, Patrones y Herramientas de Despliegue. Esto nos permitirá construir un servicio que tenga como único objetivo recuperar el modelo adecuado de nuestro almacén de modelos y realizar la inferencia solicitada. Por lo tanto, el servicio de predicción debería tener interfaces disponibles entre el panel de control y el almacén de modelos.

Además, dado que un usuario puede querer trabajar con varias combinaciones de tiendas en una misma sesión y tal vez alternar entre las previsiones de estas, deberíamos proporcionar un mecanismo para hacerlo que sea eficiente.

desviaciones en este caso, sino por solicitudes dinámicas realizadas por el usuario. Esto añade un poco de complejidad, ya que significa que nuestra solución no debe reentrenarse por cada solicitud que llegue, sino ser capaz de determinar si vale la pena reentrenar para una solicitud concreta o si el modelo ya está actualizado. Por ejemplo, si cuatro usuarios inician sesión y están mirando la misma combinación de región/tienda/artículo y todos solicitan un reentrenamiento, está bastante claro que no necesitamos reentrenar nuestro modelo cuatro veces. En su lugar, lo que debería ocurrir es que el sistema de entrenamiento registre una solicitud, realice un reentrenamiento y luego ignore de manera segura las otras solicitudes.

Existen varias formas de servir modelos de ML, como hemos discutido varias veces a lo largo de este libro. Una forma muy poderosa y flexible es envolver los modelos, o la lógica de servicio del modelo, en un servicio independiente que se limite únicamente a realizar las tareas requeridas para el servicio de la inferencia de ML. Este es el patrón de servicio que consideraremos en este capítulo y es la clásica arquitectura de “microservicios”, donde diferentes piezas de funcionalidad se dividen en sus propios servicios distintos y separados.

Esto construye resiliencia y extensibilidad en tus sistemas de software, por lo que es un gran patrón con el que es recomendable familiarizarse. Esto también es particularmente adecuado para el desarrollo de sistemas de aprendizaje automático (ML), ya que estos deben consistir en servicios de entrenamiento, inferencia y monitoreo, como se describe en el Capítulo 3, De Modelo a Fábrica de Modelos. Este capítulo explicará cómo servir un modelo de ML utilizando una arquitectura de microservicios, empleando varios enfoques con diferentes ventajas y desventajas. Luego, podrás adaptar y desarrollar estos ejemplos en tus propios proyectos futuros.

Podemos reunir estos puntos de diseño en un diagrama de diseño de alto nivel, por ejemplo, en la Figura 8.1:


![Diseñoaplicacion](figures/diseno-aplicacion.png)

La siguiente sección se centrará en llevar estas consideraciones de diseño de alto nivel a un nivel de detalle más bajo mientras realizamos la selección de herramientas antes del desarrollo.




# Seleccionando las herramientas

Ahora que tenemos un diseño de alto nivel en mente y hemos anotado algunos requisitos técnicos claros, podemos comenzar a seleccionar el conjunto de herramientas que utilizaremos para implementar nuestra solución.

Una de las consideraciones más importantes en este aspecto será qué marco utilizamos para modelar nuestros datos y construir nuestra funcionalidad de pronóstico. Dado que el problema es un problema de modelado de series temporales con necesidad de reentrenamiento y predicción rápidos, podemos considerar los pros y los contras de algunas opciones que podrían ser adecuadas antes de continuar.

| Herramienta / Framework | Pros | Contras |
|--------------------------|------|---------|
| **Sklearn** | - Ya es entendida por casi todos los científicos de datos <br> - Sintaxis muy fácil de usar <br> - Mucho soporte de la comunidad <br> - Buen soporte para ingeniería de características y creación de *pipelines* | - No tiene capacidades nativas para series temporales <br> - Requiere más ingeniería de características para aplicar a modelos de series temporales <br> - Requiere más trabajo y entendimiento por parte del ingeniero/científico |
| **Prophet** | - Enfocado únicamente en pronósticos <br> - Tiene capacidades integradas de optimización de hiperparámetros <br> - Ofrece mucha funcionalidad lista para usar <br> - Suele dar resultados precisos en una gran variedad de problemas <br> - Proporciona intervalos de confianza directamente | - No es tan comúnmente usado como Sklearn (aunque sigue siendo relativamente popular) <br> - Los métodos subyacentes son sofisticados → puede llevar a un uso de *caja negra* <br> - No es escalable de forma inherente |
| **Spark MLlib** | - Escalable de manera nativa para grandes volúmenes de datos <br> - Buen soporte para ingeniería de características y *pipelines* | - No tiene capacidades nativas para series temporales <br> - Opciones de algoritmos relativamente limitadas |

Basado en la información de la Tabla 8.2, parece que la biblioteca Prophet sería una buena opción y ofrecería un buen equilibrio entre el poder predictivo, las capacidades deseadas de series temporales y la experiencia entre los desarrolladores y científicos del equipo.


Los científicos de datos podrían entonces usar esta información para construir una prueba de concepto, con un código muy similar al mostrado en el Capítulo 1, Introducción a la Ingeniería de ML, en la sección Ejemplo 2: Pronóstico con API, que aplica Prophet a un conjunto de datos minoristas estándar.

Esto cubre el paquete de ML que utilizaremos para la modelación, pero ¿qué hay de los otros componentes? Necesitamos construir algo que permita a la aplicación frontend solicitar que el backend realice acciones, por lo que es una buena idea considerar algún tipo de framework de aplicación web. También necesitamos considerar qué sucede cuando esta aplicación backend recibe muchas solicitudes, por lo que tiene sentido construirla pensando en la escalabilidad. Otra consideración es que tenemos la tarea de entrenar no uno, sino varios modelos en este caso de uso, uno para cada tienda minorista, y por lo tanto deberíamos intentar paralelizar el entrenamiento tanto como sea posible. Las últimas piezas del rompecabezas van a ser el uso de una herramienta de gestión de modelos y la necesidad de una capa de orquestación para desencadenar trabajos de entrenamiento y monitoreo de manera programada o dinámica.

Uniendo todo esto, podemos tomar algunas decisiones de diseño sobre las herramientas de bajo nivel necesarias además del uso de la biblioteca Prophet. Podemos resumirlas en la siguiente lista:

- `Prophet`: Conocimos la biblioteca de pronósticos Prophet en el Capítulo 1, Introducción a la Ingeniería de ML. Aquí proporcionaremos un análisis más profundo de esa biblioteca y cómo funciona antes de desarrollar una canalización de entrenamiento para crear los tipos de modelos de pronóstico que vimos para ese caso de uso minorista en el primer capítulo.

- `Kubernetes`: Como se discutió en el Capítulo 6, Escalando, esta es una plataforma para orquestar múltiples contenedores a través de clústeres de cómputo y permite construir soluciones de servicio de modelos de ML altamente escalables. Usaremos esto para alojar la aplicación principal.

- `Ray Train`: Ya conocimos a Ray en el Capítulo 6, Escalando. Aquí utilizaremos Ray Train para entrenar muchos modelos de pronóstico Prophet diferentes en paralelo, y también permitiremos que estos trabajos se activen mediante una solicitud al servicio web principal que maneja las solicitudes entrantes.

- `MLflow`: Conocimos MLflow en el Capítulo 3, De Modelo a Fábrica de Modelos, y se utilizará como nuestro registro de modelos.

- `FastAPI`: Para Python, los frameworks de backend web más utilizados suelen ser Django, Flask y FastAPI. Usaremos FastAPI para crear la aplicación principal de enrutamiento del backend que servirá las predicciones e interactuará con los demás componentes de la solución. FastAPI es un framework web diseñado para ser fácil de usar y para construir aplicaciones web de alto rendimiento, y actualmente está siendo utilizado por algunas organizaciones de alto perfil, incluyendo Uber, Microsoft y Netflix (según la página principal de FastAPI).

En este capítulo, nos vamos a centrar en los componentes de este sistema que son relevantes para servir los modelos a gran escala, ya que los aspectos de entrenamiento y reentrenamiento programados se cubrirán en el Capítulo 9, Construyendo un caso de uso de Extracción, Transformación y Aprendizaje Automático. Los componentes en los que nos centramos se pueden considerar como nuestra 'capa de servicio', aunque les mostraré cómo usar Ray para entrenar varios modelos de pronóstico en paralelo. Ahora que hemos hecho algunas elecciones de herramientas, ¡vamos a empezar a construir nuestro microservicio de ML!

# Training at scale

Cuando presentamos a Ray en el Capítulo 6, Escalando, mencionamos casos de uso donde los requisitos de datos o tiempo de procesamiento eran tales que tenía sentido utilizar un marco de computación paralela muy escalable. Lo que no se hizo explícito es que, a veces, estos requisitos provienen del hecho de que en realidad queremos entrenar muchos modelos, no solo un modelo con una gran cantidad de datos o un modelo más rápidamente. Esto es lo que haremos aquí.

El ejemplo de previsión minorista que describimos en el Capítulo 1, Introducción a la Ingeniería de ML, utiliza un conjunto de datos con varias tiendas minoristas diferentes. En lugar de crear un modelo que pudiera tener un número de tienda o un identificador como característica, una mejor estrategia podría ser entrenar un modelo de previsión para cada tienda individual. Esto probablemente dará una mayor precisión, ya que las características de los datos a nivel de tienda, que pueden aportar cierto poder predictivo, no se promediarán por el modelo al observar una combinación de todas las tiendas juntas. Por lo tanto, este es el enfoque que adoptaremos, y aquí es donde podemos usar el paralelismo de Ray para entrenar varios modelos de previsión simultáneamente.

Para utilizar Ray para hacer esto, necesitamos tomar el código de entrenamiento que teníamos en el Capítulo 1 y adaptarlo ligeramente. Primero, podemos reunir las funciones que teníamos para el preprocesamiento de los datos y para el entrenamiento de los modelos de previsión. Hacer esto significa que estamos creando un proceso serial que luego podemos distribuir para ejecutarlo en la porción de los datos correspondiente a cada tienda. Las funciones originales para el preprocesamiento y el entrenamiento de los modelos eran:

In [1]:
import ray
import ray.data
import pandas as pd
from prophet import Prophet
import mlflow
import mlflow.prophet
from mlflow.client import MlflowClient
import os

# Configuración de MLflow
tracking_uri = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient(tracking_uri=tracking_uri)

# función que prepara los datos para Prophet
def prep_store_data(df: pd.DataFrame, store_id: int = 4, store_open: int = 1) -> pd.DataFrame:
    df_store = df[
        (df['Store'] == store_id) &\
        (df['Open'] == store_open)
    ].reset_index(drop=True)
    df_store['Date'] = pd.to_datetime(df_store['Date'])
    df_store.rename(columns= {'Date': 'ds', 'Sales': 'y'}, inplace=True)
    return df_store.sort_values('ds', ascending=True)

# Función que entrena el modelo Prophet y lo registra en MLflow
def train_predict(df: pd.DataFrame,
                  train_fraction: float,
                  seasonality: dict,
                  store_id: int) -> tuple:
    
    train_index = int(train_fraction * df.shape[0])
    df_train = df.copy().iloc[0:train_index]
    df_test = df.copy().iloc[train_index:]

    model = Prophet(yearly_seasonality=seasonality['yearly'],
                    weekly_seasonality=seasonality['weekly'],
                    daily_seasonality=seasonality['daily'],
                    interval_width=0.95)
    
    model.fit(df_train)
    predicted = model.predict(df_test)
    
    # REGISTRO EN MLFLOW - FORMA CORRECTA
    model_name = f"prophet-retail-forecaster-store-{store_id}"
    
    with mlflow.start_run(run_name=f"store-{store_id}"):
        # Loggear parámetros
        mlflow.log_params({
            'store_id': store_id,
            'train_fraction': train_fraction,
            'yearly_seasonality': seasonality['yearly'],
            'weekly_seasonality': seasonality['weekly'],
            'daily_seasonality': seasonality['daily'],
            'interval_width': 0.95
        })
        
        # Loggear métricas
        mlflow.log_metric("training_samples", len(df_train))
        mlflow.log_metric("test_samples", len(df_test))
        
        # Registrar el modelo - FORMA CORRECTA para MLflow >= 2.0
        mlflow.prophet.log_model(model, artifact_path="model")
        
        # Registrar el nombre del modelo manualmente
        mlflow.set_tag("model_name", model_name)
        mlflow.set_tag("store_id", store_id)
        
        print(f"✅ Modelo {model_name} registrado en MLflow")
    
    return predicted, df_train, df_test, train_index

Ahora podemos combinar todo esto en una sola función que tomará un DataFrame de pandas, preprocesará esos datos, entrenará un modelo de pronóstico Prophet y luego devolverá predicciones sobre el conjunto de prueba, el conjunto de entrenamiento, el conjunto de prueba y el tamaño del conjunto de entrenamiento, aquí etiquetado por el valor de train_index. Dado que deseamos distribuir la aplicación de esta función, necesitamos usar el decorador @ray.remote que presentamos en el Capítulo 6, Ampliando Escalamiento. Pasamos el argumento num_returns=4 al decorador para informar a Ray que esta función devolverá cuatro valores en una tupla.

In [2]:
@ray.remote(num_returns=4)  # Ahora devuelve 5 objetos (añadimos el modelo)
def prep_train_predict(df: pd.DataFrame,
                       store_id: int,
                       store_open: int = 1,
                       train_fraction: float = 0.8,
                       seasonality: dict = {'yearly': True, 'weekly': True, 'daily': False}) -> tuple:
    df = prep_store_data(df, store_id=store_id, store_open=store_open)
    return train_predict(df, train_fraction, seasonality, store_id)

Ahora que tenemos nuestra función remota, solo necesitamos aplicarla. Primero, asumimos que el conjunto de datos se ha leído en un DataFrame de pandas de la misma manera que en el Capítulo 1, Introducción a la Ingeniería de ML. La suposición aquí es que el conjunto de datos es lo suficientemente pequeño para caber en la memoria y no requiere transformaciones computacionalmente intensivas. Esto tiene la ventaja de permitirnos usar la lógica de ingestión de datos relativamente inteligente de pandas, la cual permite varios formatos de la fila de encabezado, por ejemplo, así como aplicar cualquier lógica de filtrado o transformación que queramos antes de la distribución utilizando esa sintaxis de pandas ahora familiar. Si el conjunto de datos fuera más grande o las transformaciones más intensas, podríamos haber utilizado el método ray.data.read_csv() de la API de Ray para leer los datos como un Ray Dataset. Esto lee los datos en un formato de datos Arrow, que tiene su propia sintaxis de manipulación de datos.

In [3]:
import os
file_path = lambda file: os.path.join(os.getcwd(),'data/Rossmann Store Sales', file)
df = pd.read_csv(file_path('train.csv'))
df.head()

/tmp/ipykernel_91985/4262359401.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path('train.csv'))


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


Ahora, estamos listos para aplicar nuestro entrenamiento y prueba distribuidos. Primero, podemos recuperar todos los identificadores de las tiendas del conjunto de datos, ya que vamos a entrenar un modelo para cada una.

In [4]:
store_ids = df['Store'].unique()
print(store_ids)

[   1    2    3 ... 1113 1114 1115]


Antes de hacer cualquier otra cosa, inicializaremos el clúster de Ray usando el comando ray.init() que vimos en el Capítulo 6, Ampliación. Esto evita realizar la inicialización cuando llamamos por primera vez a la función remota, lo que significa que podemos obtener tiempos precisos del procesamiento real si realizamos alguna evaluación comparativa. Para mejorar el rendimiento, también podemos usar ray.put() para almacenar el DataFrame de pandas en el almacén de objetos de Ray. Esto evita que tengamos que replicar este conjunto de datos cada vez que ejecutamos una tarea. Al poner un objeto en el almacén, se devuelve un id, que luego puedes usar como argumento de función igual que el objeto original.

In [ ]:
# Configurar experimento
mlflow.set_experiment("rossmann-store-forecasting-x")
# Inicializar Ray
ray.init(num_cpus=4)
df_id = ray.put(df) # lmacena un objeto en el “Object Store” de Ray

2025-10-14 12:45:23,130	INFO worker.py:1951 -- Started a local Ray instance.


(prep_train_predict pid=92334) 12:45:27 - cmdstanpy - INFO - Chain [1] start processing
(prep_train_predict pid=92334) 12:45:27 - cmdstanpy - INFO - Chain [1] done processing
(prep_train_predict pid=92334) 2025/10/14 12:45:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
(prep_train_predict pid=92332) 2025/10/14 12:45:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


(prep_train_predict pid=92332) ✅ Modelo prophet-retail-forecaster-store-3 registrado en MLflow
(prep_train_predict pid=92332) 🏃 View run store-3 at: http://localhost:5000/#/experiments/2/runs/c5e7cfb956984cf182a6a59cdc284585
(prep_train_predict pid=92332) 🧪 View experiment at: http://localhost:5000/#/experiments/2


(prep_train_predict pid=92333) 12:45:32 - cmdstanpy - INFO - Chain [1] start processing [repeated 39x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(prep_train_predict pid=92333) 12:45:32 - cmdstanpy - INFO - Chain [1] done processing [repeated 39x across cluster]
(prep_train_predict pid=92333) 2025/10/14 12:45:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead. [repeated 43x across cluster]
(prep_train_predict pid=92332) 2025/10/14 12:45:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature. [repeated 48x across cluster]


(prep_train_predict pid=92332) ✅ Modelo prophet-retail-forecaster-store-49 registrado en MLflow [repeated 48x across cluster]
(prep_train_predict pid=92332) 🏃 View run store-49 at: http://localhost:5000/#/experiments/2/runs/c51cd4df95bc41d7a261568a1bfe8a74 [repeated 48x across cluster]
(prep_train_predict pid=92332) 🧪 View experiment at: http://localhost:5000/#/experiments/2 [repeated 48x across cluster]


...

(prep_train_predict pid=92332) ✅ Modelo prophet-retail-forecaster-store-994 registrado en MLflow [repeated 48x across cluster]
(prep_train_predict pid=92331) 🏃 View run store-993 at: http://localhost:5000/#/experiments/2/runs/8a9ee6de6f6746ba90d305d1f4ceec68 [repeated 47x across cluster]
(prep_train_predict pid=92331) 🧪 View experiment at: http://localhost:5000/#/experiments/2 [repeated 47x across cluster]


(prep_train_predict pid=92333) 12:47:13 - cmdstanpy - INFO - Chain [1] start processing [repeated 49x across cluster]
(prep_train_predict pid=92334) 12:47:12 - cmdstanpy - INFO - Chain [1] done processing [repeated 48x across cluster]
(prep_train_predict pid=92334) 2025/10/14 12:47:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead. [repeated 50x across cluster]
(prep_train_predict pid=92333) 2025/10/14 12:47:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature. [repeated 50x across cluster]


(prep_train_predict pid=92334) ✅ Modelo prophet-retail-forecaster-store-1042 registrado en MLflow [repeated 48x across cluster]
(prep_train_predict pid=92334) 🏃 View run store-1042 at: http://localhost:5000/#/experiments/2/runs/287d09facef5416cbb32703fa9082b95 [repeated 49x across cluster]
(prep_train_predict pid=92334) 🧪 View experiment at: http://localhost:5000/#/experiments/2 [repeated 49x across cluster]


(prep_train_predict pid=92333) 12:47:18 - cmdstanpy - INFO - Chain [1] start processing [repeated 47x across cluster]
(prep_train_predict pid=92333) 12:47:18 - cmdstanpy - INFO - Chain [1] done processing [repeated 48x across cluster]
(prep_train_predict pid=92333) 2025/10/14 12:47:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead. [repeated 48x across cluster]
(prep_train_predict pid=92334) 2025/10/14 12:47:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature. [repeated 46x across cluster]


(prep_train_predict pid=92333) ✅ Modelo prophet-retail-forecaster-store-1089 registrado en MLflow [repeated 47x across cluster]
(prep_train_predict pid=92333) 🏃 View run store-1089 at: http://localhost:5000/#/experiments/2/runs/f73b1ca8112946ffbc6d3e4373372530 [repeated 47x across cluster]
(prep_train_predict pid=92333) 🧪 View experiment at: http://localhost:5000/#/experiments/2 [repeated 47x across cluster]


Ahora, necesitamos enviar nuestras tareas de Ray al clúster. Cada vez que haces esto, se devuelve una referencia a un objeto de Ray que te permitirá recuperar los datos del proceso cuando usemos ray.get para recopilar los resultados. La sintaxis que he usado aquí puede parecer un poco complicada, pero podemos desglosarla pieza por pieza. La función principal de Python, map, simplemente aplica la operación de lista a todos los elementos del resultado de la sintaxis zip. El patrón zip(*iterable) nos permite descomprimir todos los elementos en la comprensión de listas, de manera que podamos tener una lista de referencias a objetos de predicción, referencias a datos de entrenamiento, referencias a datos de prueba y finalmente referencias a índices de entrenamiento. Observa el uso de df_id para referirse al dataframe almacenado en el almacén de objetos.

In [6]:
pred_obj_refs, train_obj_refs, test_obj_refs, train_index_obj_refs = map(
    list, zip(*([prep_train_predict.remote(df_id, store_id) for store_id in store_ids])),
)

Luego necesitamos obtener los resultados reales de estas tareas, lo cual podemos hacer utilizando ray.get() como se discutió.

In [7]:

ray_results = {
    'predictions': ray.get(pred_obj_refs),
    'train_data': ray.get(train_obj_refs),
    'test_data': ray.get(test_obj_refs),
    'train_indices': ray.get(train_index_obj_refs)
}

Luego puedes acceder a los valores de estos para cada modelo con ray_results['predictions'][<index>] y así sucesivamente.

En el repositorio de Github, el archivo Chapter08/train/train_forecasters_ray.py ejecuta esta sintaxis y un ejemplo de bucle for para entrenar los modelos Prophet uno por uno de manera secuencial para comparación. Usando la biblioteca time para las mediciones y ejecutando el experimento en mi Macbook con cuatro CPUs siendo utilizadas por el clúster de Ray, pude entrenar 1,115 modelos Prophet en poco menos de 40 segundos usando Ray, en comparación con alrededor de 3 minutos y 50 segundos usando el código secuencial. ¡Eso es casi un aumento de velocidad de seis veces, sin hacer mucha optimización!

Ahora vamos a empezar a construir la capa de servicio para nuestra solución, de manera que podamos usar estos modelos de pronóstico para generar resultados para otros sistemas y usuarios.



# Sirviendo los modelos con FastAPI

El enfoque más simple y potencialmente más flexible para servir modelos de ML en un microservicio con Python es envolver la lógica de servicio dentro de una aplicación web ligera. Flask ha sido una opción popular entre los usuarios de Python durante muchos años, pero ahora el framework web FastAPI tiene muchas ventajas, lo que significa que debería considerarse seriamente como una mejor alternativa.

Algunas de las características de FastAPI que lo convierten en una excelente opción para un microservicio ligero son:

- `Validación de datos`: FastAPI utiliza y se basa en la biblioteca Pydantic, que permite hacer cumplir las indicaciones de tipo en tiempo de ejecución. Esto permite la implementación de pasos de validación de datos muy fáciles de crear que hacen que tu sistema sea mucho más robusto y ayudan a evitar comportamientos en casos extremos.

- `Flujos de trabajo asincrónicos integrados`: FastAPI te proporciona gestión de tareas asincrónicas desde el principio con las palabras clave async y await, por lo que puedes construir la lógica que necesitarás en muchos casos de manera relativamente fluida sin recurrir a bibliotecas adicionales.

- `Especificaciones abiertas`: FastAPI se basa en varios estándares de código abierto, incluyendo el estándar de API REST OpenAPI y el lenguaje declarativo JSON Schema, que ayuda a crear documentación automática de modelos de datos. Estas especificaciones ayudan a mantener el funcionamiento de FastAPI transparente y muy fácil de usar.

- `Generación automática de documentación`: El último punto mencionaba esto para los modelos de datos, pero FastAPI también genera automáticamente documentación para todo tu servicio utilizando SwaggerUI.

- `Rendimiento`: ¡Rápido está en el nombre! FastAPI utiliza el estándar Asynchronous Server Gateway Interface (ASGI), mientras que otros frameworks como Flask usan Web Server Gateway Interface (WSGI). El ASGI puede procesar más solicitudes por unidad de tiempo y lo hace de manera más eficiente, ya que puede ejecutar tareas sin esperar a que terminen las tareas anteriores. La interfaz WSGI ejecuta las tareas especificadas de manera secuencial y, por lo tanto, tarda más en procesar las solicitudes.

Entonces, las razones anteriores son por las que podría ser una buena idea usar FastAPI para servir los modelos de pronóstico en este ejemplo, pero ¿cómo vamos a hacer eso? Eso es lo que cubriremos ahora.

Cualquier microservicio debe recibir datos en algún formato específico; esto se llama la “solicitud”. Luego devolverá datos, conocidos como la “respuesta”. El trabajo del microservicio es procesar la solicitud, ejecutar una serie de tareas que la solicitud define o para las cuales proporciona entrada, crear la salida apropiada y luego transformarla en el formato de solicitud especificado. Esto puede parecer básico, pero es importante repasarlo y nos brinda el punto de partida para diseñar nuestro sistema. Está claro que tendremos que tener en cuenta los siguientes puntos en nuestro diseño:

- `Esquemas de solicitud y respuesta`: Dado que construiremos una API REST, es natural que especifiquemos el modelo de datos para las solicitudes y respuestas como objetos JSON con esquemas asociados. La clave al hacer esto es que los esquemas sean lo más simples posible y que contengan toda la información necesaria para que el cliente (el servicio que realiza la solicitud) y el servidor (el microservicio) puedan realizar las acciones apropiadas. Dado que estamos construyendo un servicio de pronóstico, el objeto de solicitud debe proporcionar suficiente información para permitir que el sistema genere un pronóstico adecuado, que la solución ascendente que llama al servicio pueda presentar a los usuarios o sobre el cual pueda realizar una lógica adicional. La respuesta deberá contener los puntos de datos del pronóstico reales o algún puntero hacia la ubicación del pronóstico.

- `Calcular`: La creación del objeto de respuesta, en este caso, una previsión, requiere cálculo, como se discute en el Capítulo 1, Introducción a la Ingeniería de ML. Una consideración clave al diseñar microservicios de ML es el tamaño de este recurso de cómputo y las herramientas adecuadas necesarias para ejecutarlo. Por ejemplo, si estás ejecutando un modelo de visión por computadora que requiere una GPU grande para realizar inferencias, no puedes hacerlo en el servidor que ejecuta el backend de la aplicación web si ese servidor es solo una máquina pequeña que utiliza CPU. De manera similar, si el paso de inferencia requiere la ingestión de un terabyte de datos, esto puede requerir el uso de un marco de paralelización como Spark o Ray ejecutándose en un clúster dedicado, que por definición tendrá que ejecutarse en máquinas diferentes de la aplicación web que sirve. Si los requisitos de cómputo son lo suficientemente pequeños y obtener datos de otra ubicación no es demasiado intenso, entonces podrías ejecutar la inferencia en la misma máquina que aloja la aplicación web.

- `Gestión de modelos`: Este es un servicio de ML, ¡así que, por supuesto, hay modelos involucrados! Esto significa, como se discutió en detalle en el Capítulo 3, De Modelo a Fábrica de Modelos, que necesitaremos implementar un proceso robusto para gestionar las versiones adecuadas de los modelos. Los requisitos de este ejemplo también implican que debemos ser capaces de utilizar muchos modelos diferentes de manera relativamente dinámica. Esto requerirá una consideración cuidadosa y el uso de una herramienta de gestión de modelos como MLflow, que también conocimos en el Capítulo 3. También debemos considerar nuestras estrategias para actualizar y revertir modelos; por ejemplo, ¿utilizaremos despliegues azul/verde o despliegues canario, como se discutió en el Capítulo 5, Patrones y Herramientas de Despliegue?

- `Monitoreo del rendimiento`: Para cualquier sistema de ML, como hemos discutido extensamente a lo largo del libro, el monitoreo del rendimiento de los modelos será críticamente importante, al igual que tomar las acciones apropiadas para actualizar o revertir estos modelos. Si los datos verdaderos para cualquier inferencia no pueden ser devueltos inmediatamente al servicio, entonces esto requerirá un proceso propio para reunir los datos verdaderos y las inferencias antes de realizar los cálculos deseados sobre ellos.

Estos son algunos de los puntos importantes que tendremos que considerar mientras construimos nuestra solución. En este capítulo, nos enfocaremos en los puntos 1 y 3, ya que el Capítulo 9 cubrirá cómo construir sistemas de entrenamiento y monitoreo que funcionen en un entorno por lotes. Ahora que sabemos algunas de las cosas que queremos tener en cuenta en nuestra solución, ¡vamos a empezar a construir!

# Esquemas de respuesta y solicitud

Si el cliente está solicitando una previsión para una tienda específica, como asumimos en los requisitos, entonces esto significa que la solicitud debería especificar algunas cosas. Primero, debería especificar la tienda, usando algún tipo de identificador de tienda que se mantenga en común entre los modelos de datos del microservicio de ML y la aplicación del cliente.

En segundo lugar, el rango de tiempo para la previsión debe proporcionarse en un formato apropiado que pueda ser fácilmente interpretado y utilizado por la aplicación. Los sistemas también deben contar con lógica para crear ventanas de tiempo de previsión adecuadas si no se proporcionan en la solicitud, ya que es perfectamente razonable asumir que si un cliente solicita “una previsión para la tienda X”, podemos asumir algún comportamiento predeterminado que proporcione una previsión para algún período de tiempo desde ahora hacia el futuro, lo cual probablemente será útil para la aplicación del cliente.

El esquema JSON de solicitud más simple que satisface esto es entonces algo como:

In [ ]:
{
"storeId": "4",
"beginDate": "2023-03-01T00:00:00Z",
"endDate": "2023-03-07T00:00:00Z"
}

Dado que este es un objeto JSON, todos los campos son cadenas, pero están poblados con valores que serán fácilmente interpretables dentro de nuestra aplicación Python. La biblioteca Pydantic también nos ayudará a hacer cumplir la validación de datos, de lo cual hablaremos más adelante. Tenga en cuenta que también debemos permitir que la aplicación cliente solicite múltiples pronósticos, por lo que debemos permitir que este JSON se extienda para permitir listas de objetos de solicitud:

In [ ]:
[
{
"storeId": "2",
"beginDate": "2023-03-01T00:00:00Z",
"endDate": "2023-03-07T00:00:00Z"
},
{
"storeId": "4",
"beginDate": "2023-03-01T00:00:00Z",
"endDate": "2023-03-07T00:00:00Z"
}
]

Como se mencionó, nos gustaría construir la lógica de nuestra aplicación de manera que el sistema aún funcione incluso si el cliente solo realiza una solicitud especificando el store_id, y luego inferimos el horizonte de pronóstico apropiado desde ahora hasta algún momento en el futuro.

Esto significa que nuestra aplicación debería funcionar cuando se envía lo siguiente como el cuerpo JSON en la llamada a la API:


In [ ]:
[
{
"storeId": "4",
}
]

Para hacer cumplir estas restricciones en la solicitud, podemos usar la funcionalidad de Pydantic donde heredamos de la BaseModel de Pydantic y creamos una clase de datos que define los requisitos de tipo que acabamos de establecer:

In [ ]:
from pydantic import BaseModel

class ForecastRequest(BaseModel):
    store_id: str
    begin_date: str | None = None
    end_date: str | None = None

Como puede ver, hemos hecho cumplir aquí que el store_id sea una cadena, pero hemos permitido que las fechas de inicio y fin del pronóstico se puedan dar como None. Si las fechas no se especifican, podríamos hacer una suposición razonable basada en nuestro conocimiento del negocio de que una ventana de tiempo útil para el pronóstico sería desde la fecha y hora de la solicitud hasta siete días después. Esto podría ser algo que se cambie o incluso se proporcione como una variable de configuración en la configuración de la aplicación. No trataremos ese aspecto en particular aquí para poder centrarnos en cosas más interesantes, por lo que esto se deja como un ejercicio divertido para el lector.

El modelo de pronóstico en nuestro caso estará basado en la biblioteca Prophet, como se discutió, y esto requiere un índice que contenga las fechas y horas para que se ejecute la previsión. Para generar esto basado en la solicitud, podemos escribir una función auxiliar sencilla:

In [ ]:
import pandas as pd
import datetime

def create_forecast_index(begin_date: str = None, end_date: str = None):
    if begin_date == None:
        begin_date = datetime.datetime.now().replace(tzinfo=None)
    else:
        begin_date = datetime.datetime.strptime(begin_date,'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=None)
    
    # Convert forecast end date
    if end_date == None:
        end_date = begin_date + datetime.timedelta(days=7)
    else:
        end_date = datetime.datetime.strptime(end_date,'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=None)

Esta lógica nos permite crear la entrada para el modelo de pronóstico una vez que se recupera de la capa de almacenamiento del modelo, en nuestro caso, MLflow.

El objeto de respuesta debe devolver el pronóstico en algún formato de datos, y siempre es imperativo que devuelvas suficiente información para que la aplicación cliente pueda asociar cómodamente el objeto devuelto con la respuesta que provocó su creación. Un esquema simple que cumpla con esto sería algo como:

In [ ]:
[
    {
        "request": {
        "store_id": "4",
        "begin_date": "2023-03-01T00:00:00Z",
        "end_date": "2023-03-07T00:00:00Z"
        },

    "forecast": [
        {

            "timestamp": "2023-03-01T00:00:00",
            "value": 20716
        },
        {
            "timestamp": "2023-03-02T00:00:00",
            "value": 20816
        },
        {
            "timestamp": "2023-03-03T00:00:00",
            "value": 21228
        },
        {
            "timestamp": "2023-03-04T00:00:00",
            "value": 21829
        },
        {
            "timestamp": "2023-03-05T00:00:00",
            "value": 21686
        },
        {
            "timestamp": "2023-03-06T00:00:00",
            "value": 22696
            },
{
"timestamp": "2023-03-07T00:00:00",
"value": 21138
}
            ]
    }
]

Permitiremos que esto se extienda como una lista de la misma manera que el esquema JSON de la solicitud. Trabajaremos con estos esquemas durante el resto de este capítulo. Ahora, veamos cómo gestionaremos los modelos en la aplicación.


# Gestion de modelos en su microservicio

En el Capítulo 3, De Modelo a Fábrica de Modelos, discutimos en detalle cómo puedes usar MLflow como una capa de almacenamiento de artefactos de modelos y metadatos en tus sistemas de ML. Haremos lo mismo aquí, así que supongamos que ya tienes un servidor de seguimiento de MLflow en funcionamiento y solo necesitamos definir nuestra lógica para interactuar con él. Si necesitas un repaso, no dudes en revisar el Capítulo 3.

Necesitaremos escribir alguna lógica que haga lo siguiente:

1. Verificar que hay modelos disponibles para su uso en producción en el servidor de MLflow.
2. Recuperar una versión del modelo que cumpla con cualquier criterio que deseemos establecer, por ejemplo, que el modelo no haya sido entrenado hace más de un número determinado de días y que tenga métricas de validación dentro de un rango seleccionado.
3. Almacenar en caché el modelo para su uso y reutilización durante la sesión de pronóstico si se desea.
4. Hace todo lo anterior para múltiples modelos si eso es requerido por el objeto de respuesta.

Para el punto 1, tendremos que tener modelos etiquetados como listos para producción en el registro de modelos de MLflow y luego podremos usar la funcionalidad MlflowClient() y mlflow pyfunc que vimos en el Capítulo 3, De Modelo a Fábrica de Modelos:


In [8]:
# Función para registrar manualmente los nombres de los modelos en el Model Registry
def register_models_in_registry(store_ids):
    """Registra los modelos en MLflow Model Registry"""
    for store_id in store_ids:
        try:
            model_name = f"prophet-retail-forecaster-store-{store_id}"
            
            # Buscar el run más reciente para este store_id
            runs = mlflow.search_runs(
                filter_string=f"tags.model_name = '{model_name}'",
                order_by=["start_time DESC"]
            )
            
            if not runs.empty:
                run_id = runs.iloc[0]["run_id"]
                
                # Registrar el modelo en el Model Registry
                mlflow.register_model(
                    model_uri=f"runs:/{run_id}/model",
                    name=model_name
                )
                
                # Transicionar a Production
                client.transition_model_version_stage(
                    name=model_name,
                    version=1,
                    stage="Production"
                )
                
                print(f"✅ Modelo {model_name} registrado en Production")
            else:
                print(f"⚠️  No se encontró run para {model_name}")
                
        except Exception as e:
            print(f"❌ Error registrando {model_name}: {e}")


Registremos los modelos para prudcción en MLflow

In [9]:
# Registrar modelos en el Model Registry
print("\n📝 Registrando modelos en MLflow Model Registry...")
register_models_in_registry(store_ids)


📝 Registrando modelos en MLflow Model Registry...


Successfully registered model 'prophet-retail-forecaster-store-1'.
2025/10/14 13:36:39 WARNING mlflow.tracking._model_registry.fluent: Run with id 6adacec91f3d496cb18a74b144f9dce1 has no artifacts at artifact path 'model', registering model based on models:/m-861c79bd486c47909f192614fe9b6214 instead
2025/10/14 13:36:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: prophet-retail-forecaster-store-1, version 1
Created version '1' of model 'prophet-retail-forecaster-store-1'.
/tmp/ipykernel_91985/1620020510.py:24: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
Successfully registe

✅ Modelo prophet-retail-forecaster-store-1 registrado en Production


2025/10/14 13:36:43 WARNING mlflow.tracking._model_registry.fluent: Run with id e828c12d9ff84c7dbd3788c16911436c has no artifacts at artifact path 'model', registering model based on models:/m-c6c399e834f449229d14e10d12f0051f instead
2025/10/14 13:36:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: prophet-retail-forecaster-store-2, version 1
Created version '1' of model 'prophet-retail-forecaster-store-2'.
Successfully registered model 'prophet-retail-forecaster-store-3'.


✅ Modelo prophet-retail-forecaster-store-2 registrado en Production


...

2025/10/14 14:47:12 WARNING mlflow.tracking._model_registry.fluent: Run with id a996737dc89b4442bdbb4bfcff31ffc4 has no artifacts at artifact path 'model', registering model based on models:/m-fa8387dbd8da49b9876785a297d843b3 instead
2025/10/14 14:47:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: prophet-retail-forecaster-store-1114, version 1
Created version '1' of model 'prophet-retail-forecaster-store-1114'.
Successfully registered model 'prophet-retail-forecaster-store-1115'.


✅ Modelo prophet-retail-forecaster-store-1114 registrado en Production


2025/10/14 14:47:16 WARNING mlflow.tracking._model_registry.fluent: Run with id 9522d665ffaa4b16834a923b9461f4a3 has no artifacts at artifact path 'model', registering model based on models:/m-fcd78a46d2414f84aa3a983beaf0ce9d instead
2025/10/14 14:47:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: prophet-retail-forecaster-store-1115, version 1


✅ Modelo prophet-retail-forecaster-store-1115 registrado en Production


Created version '1' of model 'prophet-retail-forecaster-store-1115'.


In [10]:
# Cerrar Ray
ray.shutdown()

print("🎉 Entrenamiento y registro completados!")
print(f"📊 Modelos registrados: {len(store_ids)}")

🎉 Entrenamiento y registro completados!
📊 Modelos registrados: 1115


Ya que los modelos estan MLFlow es momento de cargarlos modelos

In [14]:
import mlflow
import mlflow.pyfunc
from mlflow.client import MlflowClient
import os

# Configuración CORREGIDA - usa paréntesis en lugar de corchetes
tracking_uri = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient(tracking_uri=tracking_uri)

def get_production_model_modern(store_id: int):
    model_name = f"prophet-retail-forecaster-store-{store_id}"
    
    # Nueva forma - buscar todas las versiones
    all_versions = client.search_model_versions(f"name='{model_name}'")
    
    # Filtrar manualmente si necesitas algo específico
    production_versions = [v for v in all_versions if v.current_stage == "Production"]
    
    if production_versions:
        latest_prod = production_versions[0]
        print(f"Versión en producción: {latest_prod.version}")
        
        # Cargar el modelo
        model = mlflow.pyfunc.load_model(
            model_uri=f"models:/{model_name}/{latest_prod.version}"
        )
        return model
    else:
        print("No hay versión en producción")
        return None

Para el punto 2, podemos recuperar las métricas de un modelo determinado utilizando la funcionalidad de MLflow que describiremos a continuación. Primero, usando el nombre del modelo, se recuperan los metadatos del modelo:

In [16]:
store_id = 1
model_name = f"prophet-retail-forecaster-store-{store_id}"
all_versions = client.search_model_versions(f"name='{model_name}'")

for version in all_versions:
    print(f"Nombre: {version.name}")
    print(f"Versión: {version.version}")
    print(f"Etapa: {version.current_stage}")
    print(f"Run ID: {version.run_id}")
    print(f"Source: {version.source}")
    print(f"Creation Timestamp: {version.creation_timestamp}")
    print(f"Last Updated: {version.last_updated_timestamp}")
    print(f"Status: {version.status}")
    print(f"Description: {version.description}")
    print(f"Tags: {version.tags}")
    print("=" * 50)

Nombre: prophet-retail-forecaster-store-1
Versión: 1
Etapa: Production
Run ID: 6adacec91f3d496cb18a74b144f9dce1
Source: models:/m-861c79bd486c47909f192614fe9b6214
Creation Timestamp: 1760466999771
Last Updated: 1760466999820
Status: READY
Description: 
Tags: {}


La información de métricas para esta versión del modelo está asociada con el run_id, por lo que necesitamos obtener eso:

In [17]:
# En una sola línea (si sabes que existe versión en producción)
all_versions = client.search_model_versions(f"name='{model_name}'")
production_versions = [v for v in all_versions if v.current_stage == 'Production']

if production_versions:
    latest_model_run_id = production_versions[0].run_id
    print(f"Run ID: {latest_model_run_id}")
else:
    print("No hay versión en producción")

Run ID: 6adacec91f3d496cb18a74b144f9dce1


Si nuestro proceso de entrenamiento es orquestado por un DAG de Airflow que se ejecuta en AWS MWAA, como discutimos en el Capítulo 5, Patrones y Herramientas de Implementación, entonces el siguiente código podría usarse para invocar la canalización de entrenamiento:

In [ ]:
"""
import boto3
import http.client
import base64
import ast
# mwaa_env_name = 'YOUR_ENVIRONMENT_NAME'
# dag_name = 'YOUR_DAG_NAME'
def trigger_dag(mwaa_env_name: str, dag_name: str) -> str:
client = boto3.client('mwaa')
# get web token
mwaa_cli_token = client.create_cli_token(
Name=mwaa_env_name
)
conn = http.client.HTTPSConnection(
mwaa_cli_token['WebServerHostname']
)
mwaa_cli_command = 'dags trigger'
payload = mwaa_cli_command + " " + dag_name
headers = {
'Authorization': 'Bearer ' + mwaa_cli_token['CliToken'],
'Content-Type': 'text/plain'
}
conn.request("POST", "/aws_mwaa/cli", payload, headers)
res = conn.getresponse()
data = res.read()
dict_str = data.decode("UTF-8")
mydata = ast.literal_eval(dict_str)
return base64.b64decode(mydata['stdout']).decode('ascii')
"""

Las siguientes secciones describirán cómo se pueden unir estas partes para que el servicio FastAPI pueda envolver varios elementos de esta lógica antes de discutir cómo contenedizamos y desplegamos la aplicación.

# Uniendolo todo

Hemos definido con éxito nuestros esquemas de solicitud y respuesta, y hemos escrito la lógica para extraer el modelo adecuado de nuestro repositorio de modelos; ahora todo lo que queda por hacer es unir todo esto y realizar la inferencia real con el modelo. Hay algunos pasos para esto, que desglosaremos a continuación.

El archivo principal para el backend de FastAPI se llama app.py y contiene varias rutas diferentes de la aplicación. Durante el resto del capítulo, te mostraré las importaciones necesarias justo antes de cada fragmento de código relevante, pero el archivo real sigue la convención PEP8 de colocar las importaciones al inicio del archivo.

Primero, definimos nuestro registrador y configuramos algunas variables globales para actuar como una caché ligera en memoria de los modelos y manejadores de servicios recuperados:

In [20]:
# Logging
import logging
log_format = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
logging.basicConfig(format = log_format, level = logging.INFO)
handlers = {}
models = {}
MODEL_BASE_NAME = f"prophet-retail-forecaster-store-"

Usar variables globales para pasar objetos entre rutas de la aplicación solo es una buena idea si sabes que esta aplicación se ejecutará de manera aislada y no creará condiciones de carrera al recibir solicitudes de múltiples clientes simultáneamente. Cuando esto sucede, múltiples procesos intentan sobrescribir la variable. ¡Puedes adaptar este ejemplo para reemplazar el uso de variables globales con el uso de un caché como Redis o Memcache como un ejercicio!


Establezcamos una clase que estable la conexión a postgreSQl para obtener los modelos.


In [22]:
from mlflow.client import MlflowClient
import mlflow
from mlflow.pyfunc import PyFuncModel
from pprint import pprint

class MLFlowHandler:
    def __init__(self) -> None:
        tracking_uri = "http://127.0.0.1:5000"  # actualizado
        self.client = MlflowClient(tracking_uri=tracking_uri)
        mlflow.set_tracking_uri(tracking_uri)
    
    def check_mlflow_health(self) -> None:
        try:
            experiments = self.client.search_experiments()   
            for rm in experiments:
                pprint(dict(rm), indent=4)
            return 'Service returning experiments'
        except Exception as e:
            return f'Error calling MLFlow: {e}'
        
    def get_production_model(self, store_id: str) -> PyFuncModel:
        model_name = f"prophet-retail-forecaster-store-{store_id}"
        model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/production")
        return model

Este código se usa para inicializar, al arrancar una aplicación FastAPI, un manejador (handler) que conecta la app con MLflow —una herramienta para registrar y monitorear experimentos de machine learning— y guardarlo en una variable global (handlers). Así, cuando el servidor se inicia, la aplicación ya tiene listo el acceso a MLflow para usarlo en sus rutas o servicios sin tener que volver a configurarlo cada vez.

In [24]:
from fastapi import FastAPI
from contextlib import asynccontextmanager

#from registry.mlflow.handler import MLFlowHandler

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup - código que se ejecuta al iniciar
    await get_service_handlers()
    logging.info("Updated global service handlers")
    yield
    # Shutdown - código que se ejecuta al cerrar (opcional)
    # await cleanup_resources()

app = FastAPI(lifespan=lifespan)

async def get_service_handlers():
    mlflow_handler = MLFlowHandler()
    global handlers
    handlers['mlflow'] = mlflow_handler
    logging.info("Retreving mlflow handler {}".format(mlflow_handler))
    return handlers

Como ya se mencionó, FastAPI es excelente para soportar flujos de trabajo asíncronos, lo que permite que los recursos de computación se utilicen mientras se espera a que otras tareas se completen. La instanciación de los manejadores de servicio podría ser un proceso más lento, por lo que esto puede ser útil de adoptar aquí. Cuando se llaman funciones que usan la palabra clave async, necesitamos usar la palabra clave await, lo que significa que el resto de la función donde se ha llamado a la función async puede suspenderse hasta que se devuelva un resultado y los recursos se utilicen para otras tareas. Aquí solo tenemos un manejador para instanciar, que se encargará de las conexiones al servidor de MLflow Tracking.

Volviendo al archivo principal app.py, he escrito un pequeño endpoint de verificación de estado para obtener el estado del servicio:


In [25]:
@app.get("/health/", status_code=200)
async def healthcheck():
    global handlers
    logging.info("Got handlers in healthcheck.")
    return {
        "serviceStatus": "OK",
        "modelTrackingHealth": handlers['mlflow'].check_mlflow_health()
        }

A continuación se presenta un método para obtener el modelo de producción para un ID de tienda minorista dado. Esta función verifica si el modelo ya está disponible en la variable global (que actúa como una caché simple) y, si no está, lo agrega. Podrías ampliar este método para incluir lógica sobre la antigüedad del modelo o cualquier otro criterio que desees usar para decidir si se debe incorporar el modelo en la aplicación:

In [26]:
async def get_model(store_id: str):
    global handlers
    global models
    model_name = MODEL_BASE_NAME + f"{store_id}"
    if model_name not in models:
        models[model_name] = handlers['mlflow'].get_production_model(store_id=store_id)
    
    return models[model_name]
        

Finalmente, tenemos el endpoint de pronósticos, donde el cliente puede interactuar con esta aplicación utilizando los objetos de solicitud que definimos antes y obtener los pronósticos basados en los modelos Prophet que recuperamos de MLflow. Al igual que en el resto del libro, he omitido comentarios más largos por brevedad:

In [29]:
from pydantic import BaseModel
import datetime
import pandas as pd
 
 # REQUEST UTILITIES
class ForecastRequest(BaseModel):
    store_id: str
    begin_date: str | None = None
    end_date: str | None = None

def create_forecast_index(begin_date: str = None, end_date: str = None):
    # Convert forecast begin date
    if begin_date == None:
        begin_date = datetime.datetime.now().replace(tzinfo=None)
    else:
        begin_date = datetime.datetime.strptime(begin_date, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=None)
        
    # Convert forecast end date
    if end_date == None: 
        end_date = begin_date + datetime.timedelta(days=7)
    else:
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=None)

    forecast_index = pd.date_range(start = begin_date, end = end_date, freq = 'D')
    # Format for Prophet to consume
    return pd.DataFrame({'ds': forecast_index})

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
#from helpers.request import ForecastRequest, create_forecast_index
#from registry.mlflow.handler import MLFlowHandler
from typing import List

from contextlib import asynccontextmanager

# Logging
import logging
log_format = "%(asctime)s - %(name)s - %(levelname)s - %(message)s" 
logging.basicConfig(format = log_format, level = logging.INFO)

ml_models = {}
service_handlers = {}

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Create handlers
    service_handlers['mlflow'] = MLFlowHandler()
    logging.info("Initiatilised mlflow handler {}".format(type(service_handlers['mlflow'])))
    yield
    # Clean up handlers
    service_handlers.clear()
    ml_models.clear()
    logging.info("Handlers and ml models cleared")

#app = FastAPI(lifespan=lifespan)
app = FastAPI()



@app.get("/health/", status_code=200)
async def healthcheck():
    return {
        "serviceStatus": "OK",
        "modelTrackingHealth": service_handlers['mlflow'].check_mlflow_health(),
        "modelRegistryHealth": False
        }


@app.post("/forecast2/", status_code=200)
async def parse_request2(forecast_request: List[ForecastRequest]):
    '''
    1. Retrieve each model from model registry
    2. Forecast with it
    3. Cache it
    '''
    with lifespan(app=app):
        forecasts = []
        for item in forecast_request:
            model_name = 'prophet-retail-forecaster-store-{}'.format(item.store_id)
            if model_name not in ml_models.keys():
                ml_models[model_name] = service_handlers['mlflow'].get_production_model(item.store_id)
            else:
                pass
            forecast_input = create_forecast_index(
                begin_date=item.begin_date, 
                end_date=item.end_date
                )
            forecasts.append(ml_models[model_name].predict(forecast_input))
        return forecasts

In [35]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import nest_asyncio
#from pyngrok import ngrok
import uvicorn

# Permite reusar el loop de eventos de Jupyter
nest_asyncio.apply()

app = FastAPI()

@app.get("/health/")
async def healthcheck():
    return {"serviceStatus": "OK"}

# Ejecutar el servidor dentro de Jupyter
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [91985]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58294 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [91985]


Y puedes agregar la opción –reload si quieres desarrollar la aplicación mientras se está ejecutando. Si usas Postman (o curl o cualquier otra herramienta de tu elección) y consultas este endpoint con un cuerpo de solicitud como describimos anteriormente, ver Figura 8.2, obtendrás una salida como la que se muestra en la Figura 8.3.

Luego puedes ejecutar la aplicación localmente con:

uvicorn app:app –-host 127.0.0.1 --port 8000

¡Y eso es todo, tenemos un microservicio de ML relativamente simple que devolverá una previsión del modelo Prophet para tiendas minoristas al consultar el endpoint! Ahora pasaremos a discutir cómo podemos contenerizar esta aplicación y desplegarla en un clúster de Kubernetes para un servicio escalable.


# Contenerizar y desplegar en Kubernetes

Cuando presentamos Docker en el Capítulo 5, Patrones y Herramientas de Implementación, mostramos cómo puede usarlo para encapsular su código y luego ejecutarlo de manera consistente en muchas plataformas diferentes.

Aquí haremos esto nuevamente, pero con la idea en mente de que no solo queremos ejecutar la aplicación como un singleton en una infraestructura diferente, sino que en realidad queremos permitir que muchas réplicas diferentes del microservicio se ejecuten simultáneamente con las solicitudes siendo dirigidas de manera efectiva por un balanceador de carga. Esto significa que podemos tomar lo que funciona y hacerlo funcionar a escalas casi arbitrariamente grandes.

Haremos esto ejecutando varios pasos:

1. Conteneriza la aplicación usando Docker.
2. Sube este contenedor Docker a Docker Hub para que funcione como nuestra ubicación de almacenamiento de contenedores (puedes usar otra solución de gestión de contenedores como AWS Elastic Container Registry o soluciones similares en otro proveedor de la nube para este paso).
3. Cree un clúster de Kubernetes. Lo haremos localmente usando minikube, pero usted puede hacerlo en un proveedor de nube utilizando su servicio gestionado de Kubernetes. En AWS, esto es Elastic Kubernetes Service (EKS).
4. Define un servicio y un balanceador de carga en el clúster que pueda escalar. Aquí introduciremos el concepto de manifiestos para definir de manera programática las características de servicio y despliegue en clústeres de Kubernetes.
5. Despliega el servicio y verifica que esté funcionando como se espera.

Ahora vamos a repasar estos pasos en la siguiente sección.

## Contenerizar la aplicacion

Como se introdujo anteriormente en el libro, si queremos usar Docker, necesitamos dar instrucciones sobre cómo construir el contenedor e instalar cualquier dependencia necesaria en un Dockerfile. Para esta aplicación, podemos usar uno basado en una de las imágenes de contenedor de FastAPI disponibles, pero necesitamos obtener el archivo requeriments del entrono de trabajo. Para esto ejecutamos el comando en la terminal de visual studio code.

`pip freeze | grep -v "file://" > requirements.txt`

Posteriormente, creamos un archivo llamado Dockerfile  y dentro de el debe ir esto:

FROM tiangolo/uvicorn-gunicorn-fastapi:latest

COPY ./requirements.txt requirements.txt
RUN pip install --no-cache-dir --upgrade -r requirements.txt

COPY ./serve/src /app

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8080"]



- Se debe guardar en un archivo llamado exactamente Dockerfile (sin extensión)
- Este archivo debe estar en la carpeta raíz del proyecto (donde está el requirements.txt)

Luego, abrimo docker y abrimos una terminal en donde tenemos el proyecto y ejecutamos el siguiente comando.Donde he nombrado el contenedor como custom-forecast-service:

`docker build -t custom-forecast-service:latest .`

Nota: Es importante saber como está compuesto la carpeta de trabajo, se puede consultar con tree -L 4

Una vez que esto se haya construido exitosamente, necesitamos subirlo a Docker Hub. Puedes hacerlo iniciando sesión en Docker Hub desde la terminal y luego subiéndolo a tu cuenta ejecutando:

docker login

docker tag custom-forecast-service:latest luiscxz/custom-forecast-service:latest

`docker push luiscxz/custom-forecast-service:latest`


Esto significa que otros procesos de compilación o soluciones pueden descargar y ejecutar su contenedor.
Tenga en cuenta que antes de ingresar a Docker Hub, puede probarlo la aplicación en contenedores se ejecuta
ejecutando un comando como el siguiente, donde he incluido un indicador de plataforma para ejecutar
el contenedor localmente en mi MacBook Pro:

docker run -d --platform linux/amd64 -p 8000:8080 electricweegie/custom-

forecast-service

Ahora que hemos creado y compartido el contenedor, ahora podemos trabajar en escalarlo con un
implementación en Kubernetes.

## Scaling up with Kubernetes

Trabajar con Kubernetes puede representar una curva de aprendizaje empinada incluso para los desarrolladores más experimentados, así que aquí solo rozaremos la superficie y te daremos lo suficiente para que puedas comenzar tu propio proceso de aprendizaje. Esta sección te guiará a través de los pasos necesarios para desplegar tu microservicio de ML en un clúster de Kubernetes que se ejecute de manera local, ya que los mismos pasos se aplican para desplegarlo en un clúster remoto (con pequeñas modificaciones).

Operar clústeres de Kubernetes sin problemas en producción requiere considerar temas como redes, configuración y gestión de recursos del clúster, políticas de seguridad y mucho más. Estudiar todos estos temas en detalle requeriría un libro completo por sí mismo. De hecho, un recurso excelente para ponerse al día con muchos de estos detalles es Kubernetes in Production Best Practices de Aly Saleh y Murat Karsioglu. En este capítulo, en cambio, nos centraremos en comprender los pasos más importantes para que pueda empezar a trabajar y desarrollar microservicios de ML utilizando Kubernetes.

Primero, configurémonos para el desarrollo con Kubernetes. Aquí usaré minikube, ya que tiene algunas utilidades prácticas para configurar servicios que se pueden llamar mediante llamadas a la API REST. Anteriormente, en este libro, usé kind (Kubernetes en Docker), y puedes usarlo aquí; solo prepárate para hacer un poco más de trabajo y consultar la documentación.

Una vez que Minikube esté instalado, puedes crear tu primer clúster usando las configuraciones predeterminadas con:

minikube start

Una vez que el clúster esté en funcionamiento, puedes implementar el servicio fast-api en el clúster con el siguiente comando en la terminal:

kubectl apply -f direct-kube-deploy.yaml

donde direct-kube-deploy.yaml es un manifiesto que contiene el siguiente código:

![code](figures/code-minicube.png)

Este manifiesto define un Deployment de Kubernetes que crea y gestiona dos réplicas de una plantilla de Pod que contiene un contenedor llamado fast-api, que ejecuta la imagen de Docker que creamos y publicamos previamente, electricweegie/custom-forecast-service:latest. También define límites de recursos para los contenedores que se ejecutan dentro de los Pods y asegura que los contenedores estén escuchando en el puerto 8000.

Ahora que hemos creado un Deployment con la aplicación en él, necesitamos exponer esta solución al tráfico entrante, preferiblemente a través de un balanceador de carga para que el tráfico entrante pueda ser enrutado de manera eficiente a las diferentes réplicas de la aplicación. Para hacer esto en minikube, debes realizar algunos pasos:

1.El acceso a la red o a la máquina anfitriona no se proporciona de forma predeterminada a los Servicios que se ejecutan en el clúster de minikube, por lo que debemos crear una ruta para exponer la dirección IP del clúster utilizando el comando tunnel:

- minkube tunnel

2. Abre una nueva ventana de terminal. Esto permite que el túnel siga funcionando, y luego necesitas crear un Servicio de Kubernetes con tipo LoadBalancer que accederá al despliegue que ya hemos configurado:

- kubectl expose deployment fast-api-deployment --type=LoadBalancer --port=8080

3. Luego puedes obtener la IP externa para acceder al Servicio ejecutando:

- kubectl get svc

Esto debería generar una salida que se vea algo así:


Luego podrás usar el EXTERNAL-IP del servicio de balanceo de carga para acceder a la API, así que puedes navegar a Postman, o a tu otra herramienta de desarrollo de API, y usar http://<EXTERNAL-IP>:8080 como la URL raíz para el servicio FastAPI que ahora has construido y desplegado con éxito en Kubernetes.

## Deployment strategies

Como se discutió en el Capítulo 5, Patrones y Herramientas de Despliegue, existen varias estrategias diferentes que puedes usar para desplegar y actualizar tus servicios de ML. Hay dos componentes en esto: uno es la estrategia de despliegue para los modelos y el otro es la estrategia de despliegue para la aplicación de hosting o las canalizaciones que sirven los modelos. Ambos se pueden ejecutar en conjunto también.

Aquí discutiremos cómo tomar la aplicación que acabamos de desplegar en Kubernetes y actualizarla utilizando estrategias de despliegue canario y azul/verde. Una vez que sepas cómo hacer esto para la aplicación base, realizar una estrategia de actualización similar para los modelos se puede agregar especificando en el despliegue canario o azul/verde una versión del modelo que tenga una etiqueta apropiada. Como ejemplo, podríamos usar la etapa "staging" del registro de modelos en MLflow para obtener nuestro modelo "azul", y luego, al hacer la transición a "verde", asegurarnos de que hemos movido este modelo a la etapa "producción" en el registro de modelos utilizando la sintaxis descrita anteriormente en el capítulo y en el Capítulo 3, De Modelo a Fábrica de Modelos.

Dado que un despliegue canario es un despliegue de la nueva versión de la aplicación en un subconjunto más pequeño del entorno de producción, podemos crear un nuevo manifiesto de despliegue que haga cumplir que solo se cree y ejecute una réplica (esto podría ser más en clústeres más grandes) de la aplicación canaria. En este caso, solo se requiere que edites el número de réplicas en el manifiesto anterior a “1.”

Para garantizar que el despliegue canario sea accesible al mismo balanceador de carga, debemos utilizar el concepto de etiquetas de recursos en Kubernetes. Luego podemos desplegar un balanceador de carga que seleccione los recursos con la etiqueta deseada. A continuación se muestra un ejemplo de manifiesto para desplegar dicho balanceador de carga:

![minicube](figures/minicube.png)

O usando la misma sintaxis de minkube que se muestra arriba:

kubectl expose deployment fast-api-deployment --name=fast-api-service

--type=LoadBalancer --port=8000 --target-port=8000 --selector=app=fast-api

Después de desplegar este equilibrador de carga y el despliegue canario, puedes implementar la monitorización de los registros en el clúster o en tu modelo para determinar si el canario es exitoso y debería recibir más tráfico. En ese caso, solo tendrías que actualizar el manifiesto de despliegue para que contenga más réplicas.

Los despliegues azul/verde funcionarán de manera muy similar; en cada caso, solo debes editar el manifiesto de despliegue para etiquetar la aplicación como azul o verde. La diferencia principal entre los despliegues azul/verde y canario, sin embargo, es que el cambio del tráfico es un poco más abrupto, donde aquí podemos hacer que el servicio del balanceador de carga cambie el tráfico de producción al despliegue verde con el siguiente comando, que utiliza la CLI de kubectl para modificar la definición del selector en el servicio:

kubectl patch service fast-api-service -p
'{"spec":{"selector":{"app":"fast-api-green"}}}'

Y así es como se realizan los despliegues canary y blue/green en Kubernetes, y cómo puedes usarlos para probar diferentes versiones del servicio de pronósticos; ¡échale un vistazo!



# Resumen

En este capítulo, recorrimos un ejemplo de cómo tomar las herramientas y técnicas de los primeros siete capítulos de este libro y aplicarlas juntas para resolver un problema empresarial realista. Discutimos en detalle cómo la necesidad de un algoritmo de pronóstico activado dinámicamente puede conducir muy rápidamente a un diseño que requiere que varios servicios pequeños interactúen de manera fluida. En particular, creamos un diseño con componentes responsables de manejar eventos, entrenar modelos, almacenar modelos y realizar predicciones. Luego, explicamos cómo elegiríamos nuestro conjunto de herramientas para construir este diseño en un escenario del mundo real, considerando aspectos como la idoneidad para la tarea en cuestión, así como la familiaridad probable de los desarrolladores. Finalmente, definimos cuidadosamente las piezas clave de código que serían necesarias para construir la solución y resolver el problema de manera repetida y robusta.

En el siguiente y último capítulo, construiremos un ejemplo de un proceso de ML por lotes. Nombraremos el patrón al que esto se adhiere como Extraer, Transformar, Aprendizaje Automático, y exploraremos cuáles son los puntos clave que se deben cubrir en cualquier proyecto que tenga como objetivo construir este tipo de solución.